In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('Materiale/heart.csv')

columns_to_drop = []
for column in df.describe().columns:
    if float(abs(df['HeartDisease'].corr(df[column]))) < 0.26:
        columns_to_drop.append(column)

df = df.drop(columns_to_drop, axis = 1)

X = df.iloc[:, :-1]
y = df.iloc[:, -1].values

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Sex','ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.33, random_state=42)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
X_train

array([[-0.54791552,  0.54791552, -1.11558305, ...,  1.91287504,
        -0.67603798, -0.1060646 ],
       [-0.54791552,  0.54791552, -1.11558305, ..., -0.5227733 ,
         0.94156092, -0.82683512],
       [-0.54791552,  0.54791552, -1.11558305, ..., -0.5227733 ,
        -1.56571737, -0.82683512],
       ...,
       [-0.54791552,  0.54791552,  0.89639225, ..., -0.5227733 ,
         0.29452136,  0.43451328],
       [-0.54791552,  0.54791552,  0.89639225, ..., -0.5227733 ,
        -0.75691792, -0.82683512],
       [ 1.82509888, -1.82509888,  0.89639225, ..., -0.5227733 ,
        -0.27163825,  0.97509116]])

# Train Model

In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [13]:
y_pred = lr.predict(X_test)

In [14]:
# Confision Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[111  12]
 [ 32 148]]


0.8547854785478548

# Visualising the training set results

In [16]:
from matplotlib.colors import ListedColormap
# X_set, y_set = sc.inverse_transform(X_train), y_train
X_set, y_set = X_train, y_train 
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 10, stop = X_set[:, 0].max() + 10, step = 0.25),
                     np.arange(start = X_set[:, 1].min() - 1000, stop = X_set[:, 1].max() + 1000, step = 0.25))

plt.contourf(X1, X2, lr.predict(sc.transform(np.array([X1.ravel(), X2.ravel()]).T)).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1], c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Logistic Regression (Training set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

ValueError: X has 2 features, but StandardScaler is expecting 18 features as input.